In [1]:
from collections import defaultdict
from typing import List

import os
import pandas as pd

In [2]:
data_path = "../data/part2a"

interferences = [
    "ibench-l1i",
    "ibench-l2",
    "ibench-l1d",
    "no_interference",
    "ibench-llc",
    "ibench-membw",
    "ibench-cpu",
]
job_names = [
    "parsec-blackscholes",
    "parsec-canneal",
    "parsec-dedup",
    "parsec-ferret",
    "parsec-freqmine",
    "parsec-radix",
    "parsec-vips",
]

In [3]:
results = defaultdict(lambda: defaultdict(list))

for interference in interferences:
    for job in job_names:

        i = 0
        interference_path = f"{data_path}/{interference}/{job}"

        # Continue until the file doesn't exist
        while True:
            file_path = f"{interference_path}/job_log{i}.txt"

            if not os.path.exists(file_path):
                break

            try:
                with open(file_path) as f:
                    lines = f.readlines()
                    results[interference][job].append(lines)

                # print(f"Loaded {file_path}")
            except FileNotFoundError:
                # Alternative way to break if the file doesn't exist
                break

            i += 1

In [4]:
import re


def extract_time(lines: List[str]) -> float:

    line = [l for l in lines if "real" in l][0]

    """
    Extracts the time from a line of text.
    The line should contain the format 'real\t0m0.663s\n'.
    """
    match = re.search(r"real\s+(\d+)m([\d.]+)s", line)
    if match:
        minutes = int(match.group(1))
        seconds = float(match.group(2))
        return minutes * 60 + seconds
    else:
        raise ValueError("Line format is incorrect")

In [5]:
extracted_times = {
    interference: {
        job: extract_time(results[interference][job][0]) for job in job_names
    }
    for interference in interferences
}

In [6]:
df = pd.DataFrame(extracted_times) 
df.div(df.loc[:, "no_interference"], axis=0)

,ibench-l1i,ibench-l2,ibench-l1d,no_interference,ibench-llc,ibench-membw,ibench-cpu
parsec-blackscholes,1.582202,1.193062,1.247360,1.0,1.539970,1.361991,1.312217
parsec-canneal,1.467415,1.267095,1.236608,1.0,2.119353,1.404715,1.215996
parsec-dedup,1.971257,1.307898,1.256839,1.0,2.175802,1.600422,1.620938
parsec-ferret,2.089138,1.124242,1.143356,1.0,2.508179,1.773020,1.677633
parsec-freqmine,2.042093,1.260373,1.214472,1.0,1.908198,1.642614,2.009020
parsec-radix,1.117435,1.091896,1.088179,1.0,1.663657,1.105851,1.069230
parsec-vips,1.746591,1.572510,1.572677,1.0,1.908254,1.577104,1.625749
